# Handwriting recognition


## Data import and visualization


Import the MNIST train dataset ([https://en.wikipedia.org/wiki/MNIST_database](https://en.wikipedia.org/wiki/MNIST_database))


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# This dataset is contained in the sample data directory of Google Colab online runtimes
data = np.genfromtxt("./mnist_train_small.csv", delimiter=",")
data.shape

Store the data in a matrix and the labels in a vector


In [ ]:
labels_full = data[:, 0]
A_full = data[:, 1:].transpose()
labels_full.shape, A_full.shape

Visualize the first 30 pictures with the corresponding labels


In [ ]:
fig, axs = plt.subplots(ncols=10, nrows=3, figsize=(20, 6))
axs = axs.reshape((-1,))
for i in range(30):
    image_i = A_full[:, i].reshape((28, 28))
    axs[i].imshow(image_i, cmap="gray")
    axs[i].set_title(f"{labels_full[i]:.0f}")
    axs[i].axis("off")

Visualize the first 30 pictures corresponding the the digit "9"


In [ ]:
A_filtered = A_full[:, labels_full == 9]
fig, axs = plt.subplots(ncols=10, nrows=3, figsize=(20, 6))
axs = axs.reshape((-1,))
for i in range(30):
    image_i = A_filtered[:, i].reshape((28, 28))
    axs[i].imshow(image_i, cmap="gray")
    axs[i].axis("off")

### Classification of two digits (0 and 9)

We now focus on the digits 0 and 9 only. Build a reduced training dataset containing only images corresponding to these digits.


In [ ]:
# HINT: use a bit-mask that is true when labels are 0 OR 9 
#       "OR" might be a numpy operator, why don't you look up the documentation?
# SOLUTION-BEGIN
digits = (0, 9)
mask = np.logical_or(labels_full == digits[0], labels_full == digits[1])
A = A_full[:, mask]
labels = labels_full[mask]
# SOLUTION-END

Plot the first 30 training images.


In [ ]:
# SOLUTION-BEGIN
fig, axs = plt.subplots(nrows=3, ncols=10, figsize=(20, 6))
axs = axs.reshape((-1,))
for i in range(len(axs)):
    image_i = A[:, i].reshape((28, 28))
    axs[i].imshow(image_i, cmap="gray")
    axs[i].set_title(int(labels[i]))
    axs[i].axis("off")
# SOLUTION-END

Compute and visualize the average of the training data


In [ ]:
# SOLUTION-BEGIN
A_mean = A.mean(axis=1)
plt.imshow(A_mean.reshape((28, 28)), cmap="gray")
plt.axis("off")
# SOLUTION-END

Perform PCA on the data (SVD on the zero mean data). Then, plot the trend of

- the singular values $\sigma_k$
- the cumulate fraction of singular values: $\frac{\sum_{i=1}^{k} \sigma_i}{\sum_{i=1}^{q} \sigma_i}$
- the fraction of the "explained variance": $\frac{\sum_{i=1}^{k} \sigma_i^2}{\sum_{i=1}^{q} \sigma_i^2}$


In [ ]:
# SOLUTION-BEGIN
A_bar = A - A_mean[:, None]
U, s, VT = np.linalg.svd(A_bar, full_matrices=False)

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(18, 6))

axes[0].semilogy(s, "o-")
axes[0].set_title("Singluar values")

axes[1].plot(np.cumsum(s) / np.sum(s), "o-")
axes[1].set_title("Cumulate fraction of singular values")

axes[2].plot(np.cumsum(s**2) / np.sum(s**2), "o-")
axes[2].set_title("Explained variance")
# SOLUTION-END

Visualize the first 30 principal axes (columns of $U$)


In [ ]:
# SOLUTION-BEGIN
fig, axs = plt.subplots(nrows=3, ncols=10, figsize=(20, 6))
axs = axs.reshape((-1,))
for i in range(len(axs)):
    image_i = U[:, i].reshape((28, 28))
    axs[i].imshow(image_i, cmap="gray")
    axs[i].axis("off")
    axs[i].set_title(f"$u_{{{i + 1}}}$")
# SOLUTION-END

Compute the first two principal components corresponding to the first image


In [ ]:
# SOLUTION-BEGIN
print(f"1st principal component: {np.inner(A_bar[:, 0], U[:, 0])}" )
print(f"2nd principal component: {np.inner(A_bar[:, 0], U[:, 1])}" )
# SOLUTION-END

Define a matrix corresponding all the principal components of all the images ($U^T \bar{A}$) and check the result of the previous point

In [ ]:
# SOLUTION-BEGIN
A_pc = np.matmul(U.T, A_bar)
print(f"1st principal component: {A_pc[0, 0]}")
print(f"2nd principal component: {A_pc[1, 0]}")
# SOLUTION-END

Create a scatterplot for the first 2 principal components of all the training images, grouped by label. Consider only the first 500 pictures.


In [ ]:
# naive, slow way
# for i in range(500):
#     # take the i-th sample and find the magnitude of its projection 
#     # (that is, scalar prod) on the first and second principal axes
#     x = np.inner(A_bar[:, i], U[:, 0])
#     y = np.inner(A_bar[:, i], U[:, 1])
#     col = "r" if labels[i] == digits[0] else "b"
#     plt.scatter(x, y, marker="x", color=col, s=50)

# CHALLENGE: do the same of the above without a for loop
# SOLUTION-BEGIN
plt.scatter(A_pc[0, :500], A_pc[1, :500], marker="x", c=labels[:500], s=50)
# SOLUTION-END

## Make predictions on the test dataset
Define a classifier, by fixing a threshold on the first principal component. Namely, if a sample has the first principal component smaller than the threshold, it is in the first class, else it is in the second. Then, test the classifier on the MNIST test dataset. Compute the number of true/false positive/negative for both the digits (confusion matrix) and the accuracy of the classifier.


Choose a threshold by manual inspection, and plot the vertical line representing it

In [ ]:
threshold = 999
plt.scatter(A_pc[0, :500], A_pc[1, :500], marker="x", c=labels[:500], s=50)
plt.axvline(threshold, color="k", linestyle="--")

Load the test dataset

In [ ]:
# SOLUTION-BEGIN
data_test = np.genfromtxt("./mnist_test.csv", delimiter=",")
labels_full_test = data_test[:, 0]
A_full_test = data_test[:, 1:].transpose()
labels_full_test.shape, A_full_test.shape
# SOLUTION-END

Extract the pictures of 0s and 9s

In [ ]:
# SOLUTION-BEGIN
mask = np.logical_or(labels_full_test == digits[0], labels_full_test == digits[1])
A_test = A_full_test[:, mask]
labels_test = labels_full_test[mask]
labels_test.shape, A_test.shape
# SOLUTION-END

Plot the first 500 samples transformed following the **TRAINING** data principal components.

In [ ]:
# SOLUTION-BEGIN
# A_mean is the one of the training dataset!
A_pc_test = U.T @ (A_test - A_mean[:, None])
plt.scatter(A_pc_test[0, :500], A_pc_test[1, :500], marker="x", c=labels_test[:500], s=50)
plt.axvline(threshold, color="k", linestyle="--")
# SOLUTION-END

Compute the confusion matrix, namely how many 0s are classified as 0s, how many 0s are classified as 9s, how many 9s are classified as 0s and how many 9s are classified as 9s.

In [ ]:
PC_1 = A_pc_test[0, :]

# SOLUTION-BEGIN
labels_predicted = np.where(PC_1 > threshold, digits[0], digits[1])

# the operator "&" in numpy is equivalent to "np.logical_and"
true_0 = np.sum((labels_test == digits[0]) & (labels_predicted == digits[0]))
false_0 = np.sum((labels_test == digits[1]) & (labels_predicted == digits[0]))
true_1 = np.sum((labels_test == digits[1]) & (labels_predicted == digits[1]))
false_1 = np.sum((labels_test == digits[0]) & (labels_predicted == digits[1]))

print(f"true  {digits[0]}: {true_0}")
print(f"false {digits[0]}: {false_0}")
print(f"true  {digits[1]}: {true_1}")
print(f"false {digits[1]}: {false_1}")
accuracy = (true_0 + true_1) / (true_0 + true_1 + false_0 + false_1)
print(f"accuracy = {accuracy * 100:.2f} %")
# SOLUTION-END

You can do the same more compactly with scikit learn


In [ ]:
from sklearn import metrics

cm_display = metrics.ConfusionMatrixDisplay.from_predictions(
    labels_test, labels_predicted, cmap=plt.cm.Blues, normalize="true"
)